In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
import ast
import pickle
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
# nltk.download('omw-1.4')

#Function to pre process the given tag
def data_preprocess(txt):
    try:
        result_tag=[]
        txt=ast.literal_eval(str(txt))
        txt=' '.join(txt)        
        for tkn in gensim.utils.simple_preprocess(txt):
            if tkn not in gensim.parsing.preprocessing.STOPWORDS and len(tkn) > 3:
                result_tag.append(lemmatization_and_stemming(tkn))
        return result_tag 
    except ValueError:
        return 'NA'
    
#Applying lemmatization and stemming on the given tag    
def lemmatization_and_stemming(txt):
    stemmer_function = SnowballStemmer(language='english')
    return stemmer_function.stem(WordNetLemmatizer().lemmatize(txt, pos='v'))

In [2]:
data = pd.read_json('../dataset/games.jl',lines=True, encoding='utf-8')
data_tags = data[['tags']]
data_tags['index'] = data_tags.index

#pre processing the game tags
processed_tags = data_tags['tags'].map(data_preprocess)
processed_tags=processed_tags[processed_tags!='NA']

dict = gensim.corpora.Dictionary(processed_tags)
dict.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

#Saving dictionary
dict.save('../topic_modeling/lda_dict.dict')
bag_of_words_corpus = [dict.doc2bow(tag) for tag in processed_tags]
lda_topic_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=7, id2word=dict, passes=2, workers=2)

#Saving lda model
with open("../topic_modeling/lda_model.pkl", "wb") as f:
    pickle.dump(lda_topic_model, f)

#Printing topics and correlated words
for idx, topic in lda_topic_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

C:\Users\sk714\AppData\Local\Temp\ipykernel_17496\4017450386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tags['index'] = data_tags.index


Topic: 0 
Words: 0.115*"action" + 0.089*"platform" + 0.058*"adventur" + 0.045*"singleplay" + 0.043*"pixel" + 0.043*"graphic" + 0.033*"casual" + 0.029*"retro" + 0.028*"arcad" + 0.025*"shooter"
Topic: 1 
Words: 0.091*"horror" + 0.078*"adventur" + 0.045*"action" + 0.043*"person" + 0.040*"atmospher" + 0.036*"singleplay" + 0.035*"dark" + 0.034*"psycholog" + 0.032*"surviv" + 0.027*"violent"
Topic: 2 
Words: 0.053*"adventur" + 0.048*"rich" + 0.044*"stori" + 0.036*"anim" + 0.033*"protagonist" + 0.033*"femal" + 0.031*"singleplay" + 0.030*"choic" + 0.030*"matter" + 0.030*"novel"
Topic: 3 
Words: 0.069*"casual" + 0.058*"singleplay" + 0.052*"puzzl" + 0.042*"relax" + 0.042*"friend" + 0.040*"famili" + 0.030*"color" + 0.026*"atmospher" + 0.022*"game" + 0.021*"cute"
Topic: 4 
Words: 0.193*"simul" + 0.142*"casual" + 0.112*"strategi" + 0.067*"manag" + 0.040*"build" + 0.031*"action" + 0.023*"illustr" + 0.023*"design" + 0.022*"time" + 0.020*"sandbox"
Topic: 5 
Words: 0.081*"action" + 0.057*"strategi" + 0.